In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

#warnings.filterwarnings("ignore")
#tf version should be 2.9.0
print(tf.__version__)

2.9.0


In [2]:
rand_seed = 42

In [3]:
dir_path = r"C:\Users\acer\Desktop\Data_2D3layers\data_x"
results_dir = os.path.join(dir_path, r'logs\EfficientNet-B5')
models_dir = os.path.join(dir_path, r'models\EfficientNet-B5')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)
    
if not os.path.isdir(models_dir):
    os.makedirs(models_dir)

In [4]:
# define the location of your dataset

TRAIN_PATH = r"C:\Users\acer\Desktop\Data_2D3layers\data_x\train"
TRAIN_LABEL_PATH = r"C:\Users\acer\Desktop\Data_2D3layers\train_classification.csv"

VAL_PATH = r"C:\Users\acer\Desktop\Data_2D3layers\data_x\validation"
VAL_LABEL_PATH = r"C:\Users\acer\Desktop\Data_2D3layers\validation_classification.csv"

IMG_DIM = (224,224)
INPUT_SHAPE = (224,224,3)

BATCH_SIZE = 32
EPOCH = 20
x_axis_inc = 1 # for plotting the training acc and loss

In [5]:
import pandas as pd

train_original_df = pd.read_csv(TRAIN_LABEL_PATH)
train_original_df.head()

,ID,OA_normal_Remodel,c_erosion,c_subCyst,c_genSclerosis,c_osteophyte,c_flattening,c_corticalSclerosis
0,63-2829 L,-1,0,0,0,0,1,0
1,58-32020 R,1,1,0,0,1,1,1
2,54-25927 R,-1,0,0,0,0,1,0
3,66-218 R 2023 06 16,-1,0,0,0,0,1,0
4,60-700919 L,-1,0,0,0,0,1,1


In [6]:
def expand_dataframe(df):
    # Create a list to hold the new rows
    new_rows = []
    
    # Iterate through each row in the original dataframe
    for _, row in df.iterrows():
        # Duplicate the row 234 times (for indices 0-233)
        for i in range(224):
            new_row = row.copy()
            new_row['ID'] = f"{row['ID']}_x_{i:03d}"  # Wrap the original ID and add suffix
            one_hot_vector = np.zeros(224)
            one_hot_vector[i] = 1
            new_row['one_hot_vector'] = one_hot_vector.tolist()
            new_rows.append(new_row)
    
    # Create a new dataframe from the list of new rows
    new_df = pd.DataFrame(new_rows)
    
    return new_df

train_original_df = train_original_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

train_c_erosion_label = train_original_df[['ID','c_erosion']]
# Create the new DataFrame
train_df = expand_dataframe(train_c_erosion_label)

# Shuffle the training data
#train_df = train_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

train_df.head()

,ID,c_erosion,one_hot_vector
0,66-4559 L_x_000,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
0,66-4559 L_x_001,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
0,66-4559 L_x_002,1,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
0,66-4559 L_x_003,1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
0,66-4559 L_x_004,1,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


In [7]:
# slice_number = '160'
# train_df = train_df[train_df['ID'].str.endswith(slice_number)]
# train_df.head()

In [8]:
data_labels = train_df
target_labels = data_labels['c_erosion']
data_labels['image_path'] =  data_labels.apply(lambda row: (os.path.join(TRAIN_PATH, str(row['ID'])) + '.jpg'), axis=1)
data_labels.head()

,ID,c_erosion,one_hot_vector,image_path
0,66-4559 L_x_000,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...
0,66-4559 L_x_001,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...
0,66-4559 L_x_002,1,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...
0,66-4559 L_x_003,1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...
0,66-4559 L_x_004,1,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...


In [9]:
print(data_labels.shape)

(56896, 4)


In [10]:
import pandas as pd
import os

def check_valid_files(df, column_name='image_path'):
    # Create a new column for validity
    df['is_valid_file'] = df[column_name].apply(os.path.isfile)
    
    # Print summary
    total = len(df)
    valid = df['is_valid_file'].sum()
    invalid = total - valid
    
    print(f"Total files: {total}")
    print(f"Valid files: {valid}")
    print(f"Invalid files: {invalid}")
    
    # If there are invalid files, you can get them like this:
    if invalid > 0:
        print("\nInvalid files:")
        invalid_files = df[~df['is_valid_file']][column_name]
        for file in invalid_files:
            print(file)
    
    return df

check_valid_file_df = check_valid_files(data_labels)

# You can access the results in the DataFrame
check_valid_file_df.head(10)

Total files: 56896
Valid files: 56896
Invalid files: 0


,ID,c_erosion,one_hot_vector,image_path,is_valid_file
0,66-4559 L_x_000,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_001,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_002,1,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_003,1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_004,1,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_005,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_006,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_007,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_008,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
0,66-4559 L_x_009,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True


In [11]:
val_original_df = pd.read_csv(VAL_LABEL_PATH)
val_c_erosion_label = val_original_df[['ID','c_erosion']]
# Create the new DataFrame
val_df = expand_dataframe(val_c_erosion_label)

# Shuffle the validation data
val_df = val_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

val_df.head()

,ID,c_erosion,one_hot_vector
0,63-17877 R_x_020,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,61-5586 R_x_196,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,63-17877 R_x_090,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,60-34373 L 2019_x_113,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,53-24700 R_x_092,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [12]:
# # Filter for only the desired slice (ened with 'slice_nunber')
# val_df = val_df[val_df['ID'].str.endswith(slice_number)]

# val_df.head()

In [13]:
val_labels = val_df[['ID','c_erosion','one_hot_vector']]
target_val_labels = val_labels['c_erosion']
val_labels['image_path'] =  val_labels.apply(lambda row: (os.path.join(VAL_PATH, str(row['ID'])) + '.jpg'), axis=1)
val_labels.head()

,ID,c_erosion,one_hot_vector,image_path
0,63-17877 R_x_020,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...
1,61-5586 R_x_196,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...
2,63-17877 R_x_090,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...
3,60-34373 L 2019_x_113,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...
4,53-24700 R_x_092,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...


In [14]:
check_valid_file_df = check_valid_files(val_labels)
check_valid_file_df.head()

Total files: 12096
Valid files: 12096
Invalid files: 0


,ID,c_erosion,one_hot_vector,image_path,is_valid_file
0,63-17877 R_x_020,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True
1,61-5586 R_x_196,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True
2,63-17877 R_x_090,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True
3,60-34373 L 2019_x_113,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True
4,53-24700 R_x_092,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True


In [15]:
train_data_size = 2240
validation_data_size = 224

In [16]:
train_data = np.array([img_to_array(load_img(img, target_size=IMG_DIM))
                       for img in data_labels['image_path'][0:train_data_size].values.tolist()]).astype('float32')

In [17]:
layer_number_data = np.array([np.array(one_hot_vector) for one_hot_vector in data_labels['one_hot_vector'][0:train_data_size].values.tolist()]).astype('float32')

In [18]:
val_data = np.array([img_to_array(load_img(img, target_size=IMG_DIM))
                       for img in val_labels['image_path'][0:validation_data_size].values.tolist()]).astype('float32')

In [19]:
val_layer_number_data = np.array([np.array(one_hot_vector) for one_hot_vector in val_labels['one_hot_vector'][0:validation_data_size].values.tolist()]).astype('float32')

In [20]:
print('Training Dataset Size:', train_data.shape)
print('Validation Dataset Size:', val_data.shape)

Training Dataset Size: (2240, 224, 224, 3)
Validation Dataset Size: (224, 224, 224, 3)


## Prepare for Transfer Learning

In [21]:
prep_in = tf.keras.applications.efficientnet.preprocess_input

In [22]:
x_train_new = train_data.astype('int')
x_val_new = val_data.astype('int')
layer_number_data_new = layer_number_data.astype('int')
val_layer_number_data_new = val_layer_number_data.astype('int')

In [23]:
x_train_in = prep_in(x_train_new)
x_val_in = prep_in(x_val_new)
layer_number_data = prep_in(layer_number_data_new)
val_layer_number_data = prep_in(val_layer_number_data_new)

In [24]:
train_labels_enc = target_labels[0:train_data_size].to_numpy()
val_labels_enc = target_val_labels[0:validation_data_size].to_numpy()

In [25]:
# Get the VGG16 model so we can do transfer learning
base_model = tf.keras.applications.EfficientNetB5(input_shape=INPUT_SHAPE, include_top=False, weights='imagenet')

In [26]:
print('Number of layers in the base model: ', len(base_model.layers))

Number of layers in the base model:  577


In [27]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x0000012C3F000E20>,input_1,True
1,<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x0000012C3F195070>,rescaling,True
2,<keras.layers.preprocessing.normalization.Normalization object at 0x0000012C3F195A90>,normalization,True
3,<keras.layers.core.tf_op_layer.TFOpLambda object at 0x0000012C3F195C40>,tf.math.truediv,True
4,<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x0000012C3F195B50>,stem_conv_pad,True
...,...,...,...
572,<keras.layers.regularization.dropout.Dropout object at 0x0000012C67D0A8B0>,block7c_drop,True
573,<keras.layers.merging.add.Add object at 0x0000012C67C92AF0>,block7c_add,True
574,<keras.layers.convolutional.conv2d.Conv2D object at 0x0000012C5AF040A0>,top_conv,True
575,<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000012C67DF5FA0>,top_bn,True


## Slapping AvgPool > 1024_Dense > 512_Dense > 1_output on top of the EfficientNet

In [28]:
base_model.trainable = False

pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers[10:]]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.layers.core.activation.Activation object at 0x0000012C5AF04760>,block1a_activation,False
1,<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x0000012C67375160>,block1a_se_squeeze,False
2,<keras.layers.reshaping.reshape.Reshape object at 0x0000012C67375430>,block1a_se_reshape,False
3,<keras.layers.convolutional.conv2d.Conv2D object at 0x0000012C67105610>,block1a_se_reduce,False
4,<keras.layers.convolutional.conv2d.Conv2D object at 0x0000012C345B3760>,block1a_se_expand,False
...,...,...,...
562,<keras.layers.regularization.dropout.Dropout object at 0x0000012C67D0A8B0>,block7c_drop,False
563,<keras.layers.merging.add.Add object at 0x0000012C67C92AF0>,block7c_add,False
564,<keras.layers.convolutional.conv2d.Conv2D object at 0x0000012C5AF040A0>,top_conv,False
565,<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000012C67DF5FA0>,top_bn,False


In [29]:
fig_train = 'EfficientNetB5_A_FE_train_224'
log_file = os.path.join(results_dir, 'EfficientNetB5_A_FE_train_224.csv')
model_path = os.path.join(models_dir, 'EfficientNetB5_A_FE_224.h5')

In [30]:
# Create a new model on top

layer_number_input = tf.keras.Input(shape=(224,))
inputs = tf.keras.Input(shape=INPUT_SHAPE)
# We make sure that the base_model is running in inference mode here,
# by passing 'training=False'. This is important for fine-tuning
x = base_model(inputs, training=False)

x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='sigmoid')(x)
x = tf.keras.layers.Concatenate(axis=1)([x, layer_number_input])

x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# A Dense classifier with a single unit (binary classification)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[inputs,layer_number_input], outputs=outputs)

## Normal code

In [31]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x0000012C682BB0A0>,input_3,True
1,<keras.engine.functional.Functional object at 0x0000012C67BF5130>,efficientnetb5,False
2,<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x0000012C67E94190>,global_average_pooling2d,True
3,<keras.engine.input_layer.InputLayer object at 0x0000012C682BB100>,input_2,True
4,<keras.layers.merging.concatenate.Concatenate object at 0x0000012C67EBF460>,concatenate,True
5,<keras.layers.core.dense.Dense object at 0x0000012C67E30D00>,dense,True
6,<keras.layers.core.dense.Dense object at 0x0000012C693BBA30>,dense_1,True
7,<keras.layers.core.dense.Dense object at 0x0000012C682BBFA0>,dense_2,True


In [32]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 efficientnetb5 (Functional)    (None, 7, 7, 2048)   28513527    ['input_3[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['efficientnetb5[0][0]']         
 alAveragePooling2D)                                                                              
                                                                                              

In [33]:
callbacks = [
    CSVLogger(log_file),
    ModelCheckpoint(
        filepath=os.path.join(models_dir,"Eff-epoch-{epoch:02d}.h5"),
        save_weights_only=False,
        save_best_only=False,
        save_freq='epoch',
        verbose=1
    )
]

In [34]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), # change from_logits=True
              optimizer= keras.optimizers.RMSprop(learning_rate=1e-3),
              metrics = [keras.metrics.BinaryAccuracy()])

In [35]:
history = model.fit(x=[x_train_in,layer_number_data], 
                    y=train_labels_enc,
                    validation_data=([x_val_in,val_layer_number_data], val_labels_enc),
                    epochs=EPOCH, 
                    verbose=1, 
                    callbacks=callbacks)

Epoch 1/20
70/70 [==============================] - ETA: 0s - loss: 0.5064 - binary_accuracy: 0.7696
Epoch 1: saving model to C:\Users\acer\Desktop\Data_2D3layers\data_x\models\EfficientNet-B5\Eff-epoch-01.h5
70/70 [==============================] - 205s 3s/step - loss: 0.5064 - binary_accuracy: 0.7696 - val_loss: 1.1036 - val_binary_accuracy: 0.6071
Epoch 2/20
70/70 [==============================] - ETA: 0s - loss: 0.2863 - binary_accuracy: 0.8509
Epoch 2: saving model to C:\Users\acer\Desktop\Data_2D3layers\data_x\models\EfficientNet-B5\Eff-epoch-02.h5
70/70 [==============================] - 184s 3s/step - loss: 0.2863 - binary_accuracy: 0.8509 - val_loss: 1.2168 - val_binary_accuracy: 0.6205
Epoch 3/20
70/70 [==============================] - ETA: 0s - loss: 0.2373 - binary_accuracy: 0.8714
Epoch 3: saving model to C:\Users\acer\Desktop\Data_2D3layers\data_x\models\EfficientNet-B5\Eff-epoch-03.h5
70/70 [==============================] - 184s 3s/step - loss: 0.2373 - binary_accurac